In [2]:
import os
import PyPDF2
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI


C:\Users\PRECISION\AppData\Local\Temp\ipykernel_12628\3398718529.py:32: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\PRECISION\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Answer: The Constitution of the Islamic Republic of Pakistan, adopted on April 12, 1973, is the supreme law of the country.  It establishes Pakistan as a federal republic with Islam as the state religion.  The constitution outlines the structure of the government, including the roles of the President, Parliament (comprising the National Assembly and the Senate), the Prime Minister, the judiciary, and provincial governments.  It also guarantees fundamental rights and principles of policy, including those related to Islamic teachings and the protection of minorities.  The constitution has been amended several times since its adoption.



In [ ]:
# Import API key from config file
from config import GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [ ]:
def create_qa_system(pdf_paths):
    all_texts = ""
    for pdf_path in pdf_paths:
        # Extract text from each PDF
        pdf_text = extract_text_from_pdf(pdf_path)
        all_texts += pdf_text + "\n"  # Combine text from all PDFs

    # Split combined text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(all_texts)

    # Create embeddings
    embeddings = HuggingFaceEmbeddings()

    # Create vector store
    db = Chroma.from_texts(texts, embeddings)

    # Initialize Gemini
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
    )

    # Create a retrieval chain
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever(search_kwargs={"k": 1}))

    return qa 

In [ ]:
def main():
    pdf_paths = ["Pakistan.pdf"]  
    qa_system = create_qa_system(pdf_paths)
    
    responses = []

    while True:
        query = input("Enter your question (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break
        answer = qa_system.run(query)
        print("Answer:", answer)
        responses.append({'query': query, 'answer': answer})

if __name__ == "__main__":
    main()